In [1]:
! cp /Users/alex/Documents/romero_models/data_set/data_set_protein_sequence.fa /Users/alex/Documents/romero_models/data_set/beta_glu_DMS_log2enrichment_melted.csv . 

In [2]:
! ls 

Untitled1.ipynb
Untitled2.ipynb
beta_glu_DMS_log2enrichment_melted.csv
data.h5
data2.h5
data_set_protein_sequence.fa
diabetes_practice.ipynb
en__big_feat_set__modern.ipynb
en__big_feat_set__stock_enzyme_design.ipynb
hmm_playground.ipynb
tmp_Untitled1.ipynb
tmp_Untitled2.ipynb


In [3]:
from Bio import SeqIO

for record in SeqIO.parse('data_set_protein_sequence.fa', 'fasta'):
    seq = ''.join(record.seq)
    
seq

'MVPAAQQTAMAPDAALTFPEGFLWGSATASYQIEGAAAEDGRTPSIWDTYARTPGRVRNGDTGDVATDHYHRWREDVALMAELGLGAYRFSLAWPRIQPTGRGPALQKGLDFYRRLADELLAKGIQPVATLYHWDLPQELENAGGWPERATAERFAEYAAIAADALGDRVKTWTTLNEPWCSAFLGYGSGVHAPGRTDPVAALRAAHHLNLGHGLAVQALRDRLPADAQCSVTLNIHHVRPLTDSDADADAVRRIDALANRVFTGPMLQGAYPEDLVKDTAGLTDWSFVRDGDLRLAHQKLDFLGVNYYSPTLVSEADGSGTHNSDGHGRSAHSPWPGADRVAFHQPPGETTAMGWAVDPSGLYELLRRLSSDFPALPLVITENGAAFHDYADPEGNVNDPERIAYVRDHLAAVHRAIKDGSDVRGYFLWSLLDNFEWAHGYSKRFGAVYVDYPTGTRIPKASARWYAEVARTGVLPTAGDPNSSSVDKLAAALEHHHHHH'

In [4]:
def f(mutant_name):
    __, *pos, designed = mutant_name 
    pos0 = int(''.join(pos)) - 1 
    copy_seq = list(seq)
    copy_seq[pos0] = designed 
    copy_seq = ''.join(copy_seq)
    return copy_seq

In [5]:
len(seq)

501

In [6]:
pos_labels = ['pos_{}'.format(n) for n in range(1, len(seq)+1)] 

In [7]:
import pandas

df = pandas.read_csv('beta_glu_DMS_log2enrichment_melted.csv', index_col=0)
print(df.shape)
df.head()

(10521, 1)


,value
M1A,NaN
V2A,-0.162
P3A,-0.800
A4A,0.002
A5A,0.002


In [8]:
y = df.value.values
y

array([   nan, -0.162, -0.8  , ...,    nan,    nan,    nan])

In [9]:
my_seqs = []
for x in df.index:
    my_seq = list(f(x))
    my_seqs.append(my_seq)
    
X = pandas.DataFrame(my_seqs)
X.shape

(10521, 501)

In [10]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,491,492,493,494,495,496,497,498,499,500
0,A,V,P,A,A,Q,Q,T,A,M,...,A,A,L,E,H,H,H,H,H,H
1,M,A,P,A,A,Q,Q,T,A,M,...,A,A,L,E,H,H,H,H,H,H
2,M,V,A,A,A,Q,Q,T,A,M,...,A,A,L,E,H,H,H,H,H,H
3,M,V,P,A,A,Q,Q,T,A,M,...,A,A,L,E,H,H,H,H,H,H
4,M,V,P,A,A,Q,Q,T,A,M,...,A,A,L,E,H,H,H,H,H,H


In [11]:
from sklearn import preprocessing, feature_extraction

In [12]:
letters = list(set(X.ix[0,])) + ['*']
len(letters)

21

In [13]:
labeler = preprocessing.LabelEncoder()
labeler.fit(letters)
for col in X:
    X[col] = labeler.transform(X[col])

In [14]:
one_hot = preprocessing.OneHotEncoder()
one_hot.fit(X)
X = one_hot.transform(X)

In [15]:
type(X)

scipy.sparse.csr.csr_matrix

In [16]:
X.shape

(10521, 10521)

In [26]:
X[0:10,0:10].value

AttributeError: value not found

In [17]:
from seqlearn.hmm import MultinomialHMM

In [18]:
h = MultinomialHMM()
h.fit(X, y, [1 for n in range(len(y))])

MultinomialHMM(alpha=0.01, decode='viterbi')

In [20]:
h.predict(X[0:10,:])

array([ 0.002,  0.002,  0.002,  0.002,  0.002,  0.002,  0.002,  0.002,
        0.002,  0.002])